In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,  accuracy_score

In [2]:
iris = load_iris()
X = iris.data
y = iris.target
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(X_train.shape[0])
print(X_test.shape[0])

120
30


In [4]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

print(X_train.min())
print(X_train.max())

0.0
1.0


In [5]:
classifiers = {}
for class_label in np.unique(y_train):
    y_train_binary = (y_train == class_label).astype(int)
    clf = LogisticRegression(
        max_iter=1000,
        solver='liblinear',
        random_state=42
    )
    clf.fit(X_train, y_train_binary)
    classifiers[class_label] = clf
classifiers

{np.int64(0): LogisticRegression(max_iter=1000, random_state=42, solver='liblinear'),
 np.int64(1): LogisticRegression(max_iter=1000, random_state=42, solver='liblinear'),
 np.int64(2): LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')}

In [6]:
def predict_one_vs_all(X, classifiers):
    probabilities = np.zeros((X.shape[0], len(classifiers)))
    for class_label, clf in classifiers.items():
        probabilities[:, class_label] = clf.predict_proba(X)[:, 1]
    return probabilities, np.argmax(probabilities, axis=1)


In [7]:
# How many samples are in the test set
len(y_test)

30

In [8]:
# predicted probabilities for each class
probabilities, y_pred = predict_one_vs_all(X_test, classifiers)
probabilities[:5]

array([[0.09726567, 0.42590119, 0.36958787],
       [0.80395891, 0.15534813, 0.05265701],
       [0.00424565, 0.53837016, 0.85160244],
       [0.08761316, 0.38692913, 0.41936053],
       [0.05765313, 0.42766051, 0.46416418]])

In [9]:
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

print("Accuaracy: ", acc)

Accuaracy:  0.9


In [10]:
print(classification_report(y_test, y_pred, target_names=iris.target_names))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.67      0.80         9
   virginica       0.79      1.00      0.88        11

    accuracy                           0.90        30
   macro avg       0.93      0.89      0.89        30
weighted avg       0.92      0.90      0.90        30



In [11]:
report = classification_report(
    y_test,
    y_pred,
    target_names=iris.target_names,
    output_dict=True
)

for class_name in iris.target_names:
    if report[class_name]["recall"] == 1.0:
        print(class_name)

setosa
virginica
